# Library Imports

Importing necessary libraries and modules.

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.parse
import json

from datetime import datetime
from google.oauth2.service_account import Credentials

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Data Scraping

Defining the variables that are required to upload the data to BigQuery tables.

In [2]:
target_table = "real_estate.jakarta"
target_table_2 = "real_estate.most_recent"
project_id = "jakarta-housing-price"
credential_file = "jakarta-housing-price-595a9cff2797.json"
credential = Credentials.from_service_account_file(credential_file)
job_location = "asia-southeast2"

In [3]:
query_most_recent = pd.read_gbq(f"SELECT * FROM `{project_id}.{target_table_2}`", project_id=project_id, credentials=credential)
query_most_recent["date"] = query_most_recent["date"].dt.tz_localize(None)
query_most_recent

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent
0,2023-06-03,ANA*RUMAH BAGUS UK 10X20M 2.5LT JALAN LEGA DI ...,https://www.rumah123.com/properti/jakarta-bara...,"Jelambar, Jakarta Barat",-6.161569,106.743891,4,4,NaN,200,450,4.750000e+09,34000000.0,ANA*PROPERTY JELAMBAR


Setting up the web scraping process using Selenium. It configures the Chrome driver with specific options, defines the website URL to scrape, and initializes lists to store the scraped data.

In [4]:
# Browser Settings
website = "https://www.rumah123.com/jual/dki-jakarta/rumah/?sort=posted-desc&page=1#qid~a46c0629-67e4-410c-9c35-0c80e98987d9"
path = "chromedriver.exe"

service = Service(path)

options = Options()
options.add_argument("--headless")
options.add_argument("window-size=1920x1080")

driver = webdriver.Chrome(service=service, options=options)

# Lists to Store the Scraped Data
titles = []
links = []
locations = []
prices = []
bedrooms = []
bathrooms = []
garages = []
land_areas = []
building_areas = []
agents = []
dates = []
scraped_timestamps = []

# Iterate through Each Page
conditions_met = False

for page in range(1, 101):
    print(f"Scraping page {page}")

    url = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?sort=posted-desc&page={page}#qid~a46c0629-67e4-410c-9c35-0c80e98987d9"
    driver.get(url)

    # Property Elements
    property_elements = driver.find_elements("xpath", "//div[contains(@class, 'card-container')]")
    
    # Iterate through Each Property Element
    index = 0
    for element in property_elements:
        try:
            # Scraped Timestamp
            current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Title
            try:
                title_element = element.find_element("xpath", ".//a[@title and h2]")
                title = title_element.text
            except NoSuchElementException:
                title = float("nan")

            # Link
            try:
                link_element = element.find_element("xpath", ".//a[@title and h2]")
                link = link_element.get_attribute("href")
            except NoSuchElementException:
                link = float("nan")

            # Location
            try:
                location_element = element.find_element("xpath", ".//a[@title and h2]/following-sibling::span")
                location = location_element.text
            except NoSuchElementException:
                location = float("nan")

            # Price
            try:
                price_element = element.find_element("class name", "card-featured__middle-section__price")
                price = price_element.text
            except NoSuchElementException:
                price = float("nan")

            # Features
            features_element = element.find_element("class name", "card-featured__middle-section__attribute")

            # Bedroom
            try:
                bedroom_element = features_element.find_element("css selector", "div.ui-molecules-list__item:nth-child(1) span.attribute-text")
                bedroom = int(bedroom_element.text) if bedroom_element.text.isdigit() else float("nan")
            except NoSuchElementException:
                bedroom = float("nan")

            # Bathroom
            try:
                bathroom_element = features_element.find_element("css selector", "div.ui-molecules-list__item:nth-child(2) span.attribute-text")
                bathroom = int(bathroom_element.text) if bathroom_element.text.isdigit() else float("nan")
            except NoSuchElementException:
                bathroom = float("nan")

            # Garage
            try:
                garage_element = features_element.find_element("css selector", "div.ui-molecules-list__item:nth-child(3) span.attribute-text")
                garage = int(garage_element.text) if garage_element.text.isdigit() else float("nan")
            except NoSuchElementException:
                garage = float("nan")

            # Land Area
            try:
                land_area_element = element.find_element("xpath", ".//div[contains(text(), 'LT : ')]/span")
                land_area_text = land_area_element.text.strip()
                land_area = int(land_area_text.split()[0]) if land_area_text.split()[0].isdigit() else float("nan")
            except NoSuchElementException:
                land_area = float("nan")

            # Building Area
            try:
                building_area_element = element.find_element("xpath", ".//div[contains(text(), 'LB : ')]/span")
                building_area_text = building_area_element.text.strip()
                building_area = int(building_area_text.split()[0]) if building_area_text.split()[0].isdigit() else float("nan")
            except NoSuchElementException:
                building_area = float("nan")

            # Agent & Date
            try:
                agent_date_element = element.find_element("class name", "ui-organisms-card-r123-basic__bottom-section__agent")
                agent = agent_date_element.find_element("tag name", "p").text.strip()
                date = agent_date_element.find_elements("tag name", "p")[1].text.strip() if len(agent_date_element.find_elements("tag name", "p")) > 1 else float("nan")
            except NoSuchElementException:
                agent = float("nan")
                date = float("nan")

            # Store the Scraped Data in the Lists
            print(f"House {index + 1} (Page {page}):")

            scraped_timestamps.append(current_timestamp)
            print(f"Scraped Timestamp: {current_timestamp}")

            titles.append(title)
            print(f"Title: {title}")

            links.append(link)
            print(f"Link: {link}")

            locations.append(location)
            print(f"Location: {location}")

            prices.append(price)
            print(f"Price: {price}")

            bedrooms.append(bedroom)
            print(f"Bedroom: {bedroom}")

            bathrooms.append(bathroom)
            print(f"Bathroom: {bathroom}")

            garages.append(garage)
            print(f"Garage: {garage}")

            land_areas.append(land_area)
            print(f"Land Area: {land_area}")

            building_areas.append(building_area)
            print(f"Building Area: {building_area}")

            agents.append(agent)
            print(f"Agent: {agent}")

            dates.append(date)
            print(f"Date: {date}")

            print("--------------------")

            # Check If Conditions Are Met
            if title == query_most_recent["title"][0] and link == query_most_recent["link"][0] and \
                    location == query_most_recent["location"][0] and agent == query_most_recent["agent"][0]:
                print("CONDITIONS ARE MET")
                conditions_met = True
                break

            index += 1

        except NoSuchElementException:
            continue

    if conditions_met:
        break

driver.quit()

Scraping page 1
House 1 (Page 1):
Scraped Timestamp: 2023-06-04 09:27:22
Title: The Essence At Jakarta Garden City Tipe L6 + Attic
Link: https://www.rumah123.com/properti/jakarta-timur/hos13912771/
Location: Cakung, Jakarta Timur
Price: Rp 2,93 Miliar
Cicilan : 21 Jutaan per bulan
Bedroom: 3
Bathroom: 3
Garage: 2
Land Area: 90
Building Area: 112
Agent: Reina Irfan
Date: Diperbarui sejak 04 Juni, 2023
--------------------
House 2 (Page 1):
Scraped Timestamp: 2023-06-04 09:27:24
Title: Rumah Casa Jardin Full Renovasi Nego
Link: https://www.rumah123.com/properti/jakarta-barat/hos13912760/
Location: Green Mansion, Jakarta Barat
Price: Rp 2,8 Miliar
Cicilan : 20 Jutaan per bulan
Bedroom: 3
Bathroom: 2
Garage: 1
Land Area: 90
Building Area: 135
Agent: Kirman Wen
Date: Diperbarui sejak 04 Juni, 2023
--------------------
House 3 (Page 1):
Scraped Timestamp: 2023-06-04 09:27:24
Title: Brand new Rumah di Kemang dalam jakarta selatan
Link: https://www.rumah123.com/properti/jakarta-selatan/hos1391

Creating a dictionary called `property_details` to store the scraped property details.

In [5]:
property_details = {
    "Title": titles,
    "Link": links,
    "Location": locations,
    "Bedroom": bedrooms,
    "Bathroom": bathrooms,
    "Garage": garages,
    "Land (m²)": land_areas,
    "Building (m²)": building_areas,
    "Price": prices,
    "Agent": agents,
    "Date": dates
}

for key, value in property_details.items():
    print("Key:", key)
    print("Length of Values:", len(value))
    print("---")

Key: Title
Length of Values: 333
---
Key: Link
Length of Values: 333
---
Key: Location
Length of Values: 333
---
Key: Bedroom
Length of Values: 333
---
Key: Bathroom
Length of Values: 333
---
Key: Garage
Length of Values: 333
---
Key: Land (m²)
Length of Values: 333
---
Key: Building (m²)
Length of Values: 333
---
Key: Price
Length of Values: 333
---
Key: Agent
Length of Values: 333
---
Key: Date
Length of Values: 333
---


# Data Cleaning

Defining a dictionary called `month_mapping` that maps Indonesian month names to their English equivalents.

In [6]:
month_mapping = {
    "(?i)Januari": "January",
    "(?i)Februari": "February",
    "(?i)Maret": "March",
    "(?i)April": "April",
    "(?i)Mei": "May",
    "(?i)Juni": "June",
    "(?i)Juli": "July",
    "(?i)Agustus": "August",
    "(?i)September": "September",
    "(?i)Oktober": "October",
    "(?i)November": "November",
    "(?i)Desember": "December"
}

Defining a function `convert_price` to convert the scraped price string into a numeric value.

In [7]:
def convert_price(price):
    number_split = price.split(" ")

    numeric = float(number_split[0].replace(",", "."))
    suffix = number_split[1]

    if "triliun" in suffix.lower():
        multiplier = 10**12
    elif "miliar" in suffix.lower():
        multiplier = 10**9
    elif "juta" in suffix.lower():
        multiplier = 10**6
    else:
        multiplier = 1

    numeric *= multiplier
    return numeric

Retrieving the latitude and longitude coordinates for specific locations in Jakarta using the Nominatim API and storing them in a dictionary. The dictionary is then saved as a JSON file.

In [8]:
jakarta_locations = ["Jakarta Pusat", "Jakarta Utara", "Jakarta Timur", "Jakarta Selatan", "Jakarta Barat"]

coordinates = {}

for loc in jakarta_locations:
    url = "https://nominatim.openstreetmap.org/search/" + urllib.parse.quote(loc) +"?format=json"
    response = requests.get(url).json()
    coordinates[loc] = {"latitude": response[0]["lat"], "longitude": response[0]["lon"]}

with open("jakarta_coordinates.json", "w") as file:
    file.write(json.dumps(coordinates))

Loading the JSON file containing the coordinates of Jakarta locations into a dictionary.

In [9]:
with open("jakarta_coordinates.json", "r") as file:
    coordinates = json.loads(file.read())

coordinates

{'Jakarta Pusat': {'latitude': '-6.18233995',
  'longitude': '106.84287153600738'},
 'Jakarta Utara': {'latitude': '-6.136197', 'longitude': '106.90069022428446'},
 'Jakarta Timur': {'latitude': '-6.26289085',
  'longitude': '106.88222894692834'},
 'Jakarta Selatan': {'latitude': '-6.28381815',
  'longitude': '106.80486325728526'},
 'Jakarta Barat': {'latitude': '-6.161569', 'longitude': '106.74389053715214'}}

Creating Pandas DataFrames from the scraped data. It performs various data cleaning and transformation steps such as replacing text, converting data types, applying the `convert_price` function, retrieving latitude and longitude coordinates from the `coordinates` dictionary, and reordering the columns.

In [10]:
df = pd.DataFrame({
    "Title": titles,
    "Link": links,
    "Location": locations,
    "Bedroom": bedrooms,
    "Bathroom": bathrooms,
    "Garage": garages,
    "Land m2": land_areas,
    "Building m2": building_areas,
    "Price": prices,
    "Agent": agents,
    "Date": dates,
    "Scraped Timestamp": scraped_timestamps
})

df["Date"] = df["Date"].str.replace("Diperbarui sejak ", "").str.replace(",", "")
df["Date"] = df["Date"].replace(month_mapping, regex=True)
df["Date"] = pd.to_datetime(df["Date"])

df["Price IDR"] = df["Price"].str.split("\n").str[0].str.replace("Rp ", "")
df["Price IDR"] = df["Price IDR"].apply(convert_price)

df["Monthly Payment IDR"] = df["Price"].str.split("\n").str[1].str.replace("Cicilan : ", "").str.replace(" per bulan", "")
df["Monthly Payment IDR"] = df["Monthly Payment IDR"].apply(convert_price)

df["Latitude"] = df["Location"].str.split(", ").str[-1].map(lambda x: coordinates.get(x, {}).get("latitude"))
df["Longitude"] = df["Location"].str.split(", ").str[-1].map(lambda x: coordinates.get(x, {}).get("longitude"))

df["Scraped Timestamp"] = pd.to_datetime(df["Scraped Timestamp"])

df = df.drop("Price", axis=1)
df = df[["Date", "Title", "Link", "Location", "Latitude", "Longitude", "Bedroom", "Bathroom", "Garage", "Land m2", "Building m2", "Price IDR", "Monthly Payment IDR", "Agent", "Scraped Timestamp"]]
df.columns = df.columns.str.lower().str.replace(" ", "_")

for col in ["latitude", "longitude", "bedroom", "bathroom", "garage", "land_m2", "building_m2", "price_idr", "monthly_payment_idr"]:
    df[col] = df[col].astype(float)

df = df.drop_duplicates(subset=["title", "location", "bedroom", "bathroom", "garage", "land_m2", "building_m2"]).reset_index(drop=True)

condition = (
    (df["title"] == query_most_recent["title"][0]) &
    (df["link"] == query_most_recent["link"][0]) &
    (df["location"] == query_most_recent["location"][0]) &
    (df["agent"] == query_most_recent["agent"][0])
)
df = df[~condition]

df.head()

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent,scraped_timestamp
0,2023-06-04,The Essence At Jakarta Garden City Tipe L6 + A...,https://www.rumah123.com/properti/jakarta-timu...,"Cakung, Jakarta Timur",-6.262891,106.882229,3.0,3.0,2.0,90.0,112.0,2.930000e+09,21000000.0,Reina Irfan,2023-06-04 09:27:22
1,2023-06-04,Rumah Casa Jardin Full Renovasi Nego,https://www.rumah123.com/properti/jakarta-bara...,"Green Mansion, Jakarta Barat",-6.161569,106.743891,3.0,2.0,1.0,90.0,135.0,2.800000e+09,20000000.0,Kirman Wen,2023-06-04 09:27:24
2,2023-06-04,Brand new Rumah di Kemang dalam jakarta selatan,https://www.rumah123.com/properti/jakarta-sela...,"Kemang, Jakarta Selatan",-6.283818,106.804863,4.0,4.0,2.0,500.0,600.0,2.900000e+10,207000000.0,Rizora Muhamad,2023-06-04 09:27:24
3,2023-06-04,Rumah Cantik di Kebayoran Residence Bintaro Jaya,https://www.rumah123.com/properti/jakarta-bara...,"Puri Indah, Jakarta Barat",-6.161569,106.743891,NaN,NaN,NaN,72.0,65.0,1.700000e+09,12000000.0,Yanuar Abidin,2023-06-04 09:27:25
4,2023-06-04,Rumah Cantik di Kebayoran Residence Bintaro Jaya,https://www.rumah123.com/properti/jakarta-sela...,"Pondok Labu, Jakarta Selatan",-6.283818,106.804863,NaN,NaN,NaN,72.0,65.0,1.700000e+09,12000000.0,Yanuar Abidin,2023-06-04 09:27:25


In [11]:
most_recent = df[df["scraped_timestamp"] == df["scraped_timestamp"].min()]
most_recent

,date,title,link,location,latitude,longitude,bedroom,bathroom,garage,land_m2,building_m2,price_idr,monthly_payment_idr,agent,scraped_timestamp
0,2023-06-04,The Essence At Jakarta Garden City Tipe L6 + A...,https://www.rumah123.com/properti/jakarta-timu...,"Cakung, Jakarta Timur",-6.262891,106.882229,3.0,3.0,2.0,90.0,112.0,2.930000e+09,21000000.0,Reina Irfan,2023-06-04 09:27:22


# Data Storage

Uploading the DataFrames to the specified BigQuery tables.

In [12]:
schema = [
    {"name": "date", "type": "DATE"},
    {"name": "title", "type": "STRING"},
    {"name": "link", "type": "STRING"},
    {"name": "location", "type": "STRING"},
    {"name": "latitude", "type": "FLOAT64"},
    {"name": "longitude", "type": "FLOAT64"},
    {"name": "bedroom", "type": "FLOAT64"},
    {"name": "bathroom", "type": "FLOAT64"},
    {"name": "garage", "type": "FLOAT64"},
    {"name": "land_m2", "type": "FLOAT64"},
    {"name": "building_m2", "type": "FLOAT64"},
    {"name": "price_idr", "type": "FLOAT64"},
    {"name": "monthly_payment_idr", "type": "FLOAT64"},
    {"name": "agent", "type": "STRING"}
]

In [13]:
df_without_timestamp = df.copy()
df_without_timestamp = df_without_timestamp.drop("scraped_timestamp", axis=1)

df_without_timestamp.to_gbq(
    destination_table=target_table,
    project_id=project_id,
    if_exists="append",
    location=job_location,
    chunksize=10_000,
    progress_bar=True,
    credentials=credential,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [14]:
most_recent_without_timestamp = most_recent.copy()
most_recent_without_timestamp = most_recent_without_timestamp.drop("scraped_timestamp", axis=1)

most_recent_without_timestamp.to_gbq(
    destination_table=target_table_2,
    project_id=project_id,
    if_exists="replace",
    location=job_location,
    progress_bar=True,
    credentials=credential,
    table_schema=schema
)

100%|██████████| 1/1 [00:00<?, ?it/s]
